# Case Study 2: Exploratory Data Analysis
This notebook was an initial playground for testing a variety of code.  Much of the mistakes were deleted, but the notebook does go over some of the design choices I made.

Case Study #2:  
There is 1 dataset(csv) with 3 years worth of customer orders. There are 4 columns in the csv dataset: index, CUSTOMER_EMAIL(unique identifier as hash), Net_Revenue, and Year.  
__For each year we need the following information__:
* Total revenue for the current year  
* New Customer Revenue e.g. new customers not present in previous year only  
* Existing Customer Growth, Revenue of existing customers for current year – Revenue of existing customers from prior year  
* Revenue lost from attrition 
* Existing Customer Revenue Current Year  
* Existing Customer Revenue Prior Year  
* Total Customers Current Year  
* Total Customers Previous Year  
* New Customers  
* Lost Customers  

Dataset
https://www.dropbox.com/sh/xhy2fzjdvg3ykhy/AADAVKH9tgD_dWh6TZtOd34ia?dl=0
customer_orders.csv

Output
An HTML website with the results of the data. Please highlight which year the calculations are for. All code should be hosted on GitHub for viewing. Please provide URL’s to both the output and the GitHub repo.

In [1]:
import pandas as pd
import io

## Import dataset

In [2]:
# Import from local system
from google.colab import files
uploaded = files.upload()

Saving casestudy.csv to casestudy.csv


In [3]:
df_full = pd.read_csv(io.BytesIO(uploaded['casestudy.csv']))
print(df_full.columns)
df_full.head()

Index(['Unnamed: 0', 'customer_email', 'net_revenue', 'year'], dtype='object')


,Unnamed: 0,customer_email,net_revenue,year
0,0,nhknapwsbx@gmail.com,249.92,2015
1,1,joiuzbvcpn@gmail.com,87.61,2015
2,2,ukkjctepxt@gmail.com,168.38,2015
3,3,gykatilzrt@gmail.com,62.40,2015
4,4,mmsgsrtxah@gmail.com,43.08,2015


## Analysis

Each calculation has been made into it's own function.  The reason for this is to follow the __single responsibility principle__ from SOLID principles of object oriented programming.  While we are not defining any classes in this program, it is still important to define functions to have a single purpose. 

There are many reasons as to why we would want for this to be true.  First off, it makes testing easier.  Since each function only performs one operation, unit testing for each function is easier.  Also, writing your code this way make it more dynamic.  Less things are hard-coded and everything is more modular.  

In [4]:
# Return a dataframe from the year's data
def data_for_year(df,year):
  new_df = df.loc[df['year'] == year]
  print(type(year))
  return new_df

# Get sorted list of unique years in full dataset
# Smallest year is in position zero
def years_in_data(df):
  years = sorted(set(df['year']))
  return years 

# Input parameters: current years data (dataframe), previous year's data 
# (dataframe), type of join (string of inner, outer, left, or right), 
def merged_data(df_current, df_prior, how, joined_type):
  df_merged = pd.merge(df_current,df_prior, how = how, on = ['customer_email']
                       ,indicator=True)
  df = df_merged.loc[df_merged['_merge'] == joined_type]
  return df

# Returns the total revenue for the given year
def total_revenue(df, year):
  sub_df = data_for_year(df, year)
  total_revenue = round(sum(sub_df['net_revenue']),2)
  return total_revenue

# Returns revunue made from new customers.  Only compares with the previous year
def new_customer_revenue(df, year):
  years = years_in_data(df)
  if years[0] == year:
    return "N/A"
  else:
    df_year_current = data_for_year(df, year)
    df_year_prior = data_for_year(df, year - 1)
    df_unique = merged_data(df_year_current, df_year_prior, 'left', 'left_only')
    return sum(df_unique['net_revenue_x'])

# Returns revenue of existing customers from current year
def existing_customer_revenue_current_year(df, year):
  years = years_in_data(df)
  if years[0] == year:
    return "N/A"
  else:
    df_year_current = data_for_year(df, year)
    df_year_prior = data_for_year(df, year - 1)
    df_common = merged_data(df_year_current, df_year_prior, 'inner', 'both')
    return sum(df_common['net_revenue_x'])

# Returns revenue of existing customers from prior year
def existing_customer_revenue_prior_year(df, year):
  years = years_in_data(df)
  if years[0] == year:
    return "N/A"
  else:
    df_year_current = data_for_year(df, year)
    df_year_prior = data_for_year(df, year - 1)
    df_common = merged_data(df_year_current, df_year_prior, 'inner', 'both')
    return sum(df_common['net_revenue_y'])

# Revenue of existing customers for current year minus 
# Revenue of existing customers from prior year  
def existing_customer_growth(df, year):
  years = years_in_data(df)
  if years[0] == year:
    return "N/A"
  else:
    return existing_customer_revenue_current_year(df, year) - existing_customer_revenue_prior_year(df, year)

# Calculates the revnue lost from customers who did not make any purchases 
# this year.  
def revenue_lost_from_attrition(df, year):
  years = years_in_data(df)
  if years[0] == year:
    return "N/A"
  else:
    df_year_current = data_for_year(df, year)
    df_year_prior = data_for_year(df, year - 1)
    df_unique = merged_data(df_year_current, df_year_prior,'right','right_only')
    return sum(df_unique['net_revenue_y'])

# Returns the total number of customer from a given year
def total_customers_by_year(df, year):
  df_year_current = data_for_year(df, year)
  unique_emails = set(df_year_current['customer_email'])
  return len(unique_emails)

# Returns a list of customers who did not make any puchases last year but
# did make purchases this year
def new_customers(df, year):
  years = years_in_data(df)
  if years[0] == year:
    return "N/A"
  else:
    df_year_current = data_for_year(df, year)
    df_year_prior = data_for_year(df, year - 1)
    df_unique = merged_data(df_year_current, df_year_prior, 'left', 'left_only')
    return df_unique['customer_email'].tolist()

# Retuns a list of all the customers who did not make any purchases this year
def lost_customers(df, year):
  years = years_in_data(df)
  if years[0] == year:
    return "N/A"
  else:
    df_year_current = data_for_year(df, year)
    df_year_prior = data_for_year(df, year - 1)
    df_unique = merged_data(df_year_current, df_year_prior,'right','right_only')
    return df_unique['customer_email'].tolist()

In [5]:
data_for_year(df_full,2015)

<class 'int'>


,Unnamed: 0,customer_email,net_revenue,year
0,0,nhknapwsbx@gmail.com,249.92,2015
1,1,joiuzbvcpn@gmail.com,87.61,2015
2,2,ukkjctepxt@gmail.com,168.38,2015
3,3,gykatilzrt@gmail.com,62.40,2015
4,4,mmsgsrtxah@gmail.com,43.08,2015
...,...,...,...,...
231289,231289,xtrpmgjbwp@gmail.com,216.89,2015
231290,231290,peeorxpsbr@gmail.com,39.16,2015
231291,231291,vanasezjpw@gmail.com,233.46,2015
231292,231292,dnpremlztb@gmail.com,136.27,2015
